# Comparing the best and the worst alignement for grids

How sensitive are the grids to missalignment.
We compare the best alignment of a grid to its worst.

In [40]:
from bisect import bisect

from aemeasure import read_as_pd
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

In [41]:
from _utils import convert_to_percentage_interval

from pcpptc import PolygonInstance, Solution
from pcpptc.plot import setup_plot, plot_polygon_instance, plot_solution
from pcpptc.plot.polgyon_instance import plot_coverage

In [42]:
read_as_pd("./solutions/1.results.json").columns

Loaded dataframe ./solutions/1.results.json
Executed on: ['algry01' 'algry02']
During: 2021-05-21 14:37:25.626364 and 2021-06-01 01:37:18.179121


Index(['instance', 'instance_path', 'runtime', 'timestamp', 'hostname', 'argv',
       'git_revision', 'solver', 'solution', 'i', 'coverage', 'touring_cost',
       'length', 'turn_sum', 'turn_factor'],
      dtype='object')

In [53]:
tables = []
for f in os.listdir("./solutions"):
    if not f.endswith(".results.json"):
        continue
    f = os.path.join("./solutions", f)
    tables.append(read_as_pd(f))

data = pd.concat(tables, ignore_index=True)
# data.dropna(inplace=True)

Loaded dataframe ./solutions/75.results.json
Executed on: ['algry01' 'algry02']
During: 2021-05-21 09:56:32.369542 and 2021-05-30 23:12:51.253141
Loaded dataframe ./solutions/156.results.json
Executed on: ['algry01' 'algry02']
During: 2021-05-21 15:40:10.168809 and 2021-06-02 05:39:33.243133
Loaded dataframe ./solutions/119.results.json
Executed on: ['algry01' 'algry02']
During: 2021-05-21 12:59:00.092247 and 2021-05-31 04:52:30.695564
Loaded dataframe ./solutions/173.results.json
Executed on: ['algry01' 'algry02']
During: 2021-05-21 16:46:18.925022 and 2021-06-01 05:44:43.928209
Loaded dataframe ./solutions/144.results.json
Executed on: ['algry01' 'algry02']
During: 2021-05-21 10:03:49.638596 and 2021-05-30 23:25:05.968252
Loaded dataframe ./solutions/185.results.json
Executed on: ['algry01' 'algry02']
During: 2021-05-21 19:44:53.863266 and 2021-06-02 03:48:34.655858
Loaded dataframe ./solutions/154.results.json
Executed on: ['algry01' 'algry02']
During: 2021-05-21 10:35:54.058267 and

In [54]:
def descr(solver):
    if "hex" in solver.lower():
        type = "Hexagonal"
    elif "square" in solver.lower():
        type = "Square"
    else:
        raise ValueError(f"Could not detect type of {solver}")
    if "pb=True" in solver or "point_based=True" in solver:
        d = "Point-based"
    elif "pb=False" in solver or "point_based=False" in solver:
        d = "Line-based"
    else:
        raise ValueError(f"Could not detect base of {solver}")
    return f"{type}, {d}"


data["Grid"] = data["solver"].apply(descr)

data.groupby("Grid")["instance"].nunique()

Grid
Hexagonal, Line-based     200
Hexagonal, Point-based    139
Square, Line-based        200
Square, Point-based       139
Name: instance, dtype: int64

In [ ]:
data.dropna(inplace=True)

In [46]:
# add instance area
instance_paths = list(data["instance_path"].unique())
instances = [PolygonInstance.from_json(file_path=f) for f in instance_paths]
instance_areas = pd.DataFrame(
    data={"area": [i.original_area.area for i in instances]}, index=instance_paths
)
data = data.merge(instance_areas, left_on="instance_path", right_index=True)
data["relative_coverage"] = 100 * data["coverage"] / data["area"]

In [47]:
best_aligned = data.groupby(["instance", "Grid"])[["touring_cost"]].min()
worst_aligned = data.groupby(["instance", "Grid"])[["touring_cost"]].max()
t = best_aligned.merge(
    right=worst_aligned, suffixes=("_min", "_max"), left_index=True, right_index=True
).reset_index()
t["diff"] = (100 * t["touring_cost_max"] / t["touring_cost_min"]) - 100
t

,instance,Grid,touring_cost_min,touring_cost_max,diff
0,0,"Hexagonal, Line-based",458.543796,691.884979,50.887436
1,0,"Square, Line-based",489.389372,869.187125,77.606457
2,1,"Hexagonal, Line-based",605.740807,1008.015307,66.410335
3,1,"Hexagonal, Point-based",818.265721,1112.097697,35.909115
4,1,"Square, Line-based",563.911486,1478.412496,162.171020
...,...,...,...,...,...
673,98,"Square, Line-based",14087.698328,22989.850828,63.190965
674,99,"Hexagonal, Line-based",3428.514300,4751.984464,38.601856
675,99,"Hexagonal, Point-based",4525.093461,4797.503462,6.019986
676,99,"Square, Line-based",3503.858377,6833.882812,95.038785


In [48]:
# add instance information

instance_data = data.groupby(["instance"])[["area", "turn_factor"]].min()
t = t.merge(right=instance_data, left_on="instance", right_index=True)
t.sort_values(["Grid"], inplace=True)

In [49]:
data_area_interval = convert_to_percentage_interval(t, on_column="area", percentage=5)
data_area_interval.sort_values(by=["Grid"], ascending=False, inplace=True)


def rename(x):
    return x if "Square" in x else x.replace("Hexagonal", "Triangular")


data_area_interval["Grid"] = data_area_interval["Grid"].apply(rename)
t["Grid"] = t["Grid"].apply(rename)

In [ ]:
plt.figure(figsize=(5, 3.5))
sns.lineplot(data=data_area_interval, x="area", y="diff", hue="Grid")
plt.legend(loc="upper right")
plt.ylabel("Difference of touring cost (%)")
plt.xlabel("Area ±5%")
plt.title("Lower is better")
plt.legend(loc="upper left")
plt.tight_layout()
plt.savefig("././output/05_best_vs_worst_area.pdf")
plt.show()

In [ ]:
plt.figure(figsize=(5, 3.5))
sns.boxplot(data=t, x="turn_factor", y="diff", hue="Grid")
plt.ylabel("Difference of touring cost (%)")
plt.xlabel("Optimization Weight of Turn-Cost")
plt.title("Lower is better")
plt.legend(loc="upper right")
# plt.gca().get_legend().remove()
plt.tight_layout()
plt.savefig("././output/05_best_vs_worst_turn_factor.pdf")
plt.show()

In [50]:
data.groupby(["Grid"])["instance"].nunique()

Grid
Hexagonal, Line-based     200
Hexagonal, Point-based    139
Square, Line-based        200
Square, Point-based       139
Name: instance, dtype: int64